# Serve Zone Distribution

function to classify zones of each serve and calculate win percentage in each zone

creates a json file that can be passed into analytics html to create visual

In [2]:
import pandas as pd
import numpy as np

In [3]:
def classify_zone(df):
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            return 'Ad Wide'
        else:
            return 'Deuce Wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            return 'Ad Body'
        else:
            return 'Deuce Body'
    elif -52.5 < x < 52.5:
        if sign > 0:
            return 'Ad T'
        else:
            return 'Deuce T'
    else:
        return np.nan

In [4]:
def serve_zone_distribution(file_path):
    df_shots = pd.read_excel(file_path, sheet_name='Shots')
    df_points = pd.read_excel(file_path, sheet_name='Points')
    
    # only use matches with complete data
    df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])]

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')
    
    serves = combined[combined['Stroke'] == 'Serve']
    serves_in = serves[serves['Result'] == 'In']

    # zone classification
    serves_in['x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in['y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in['Zone'] = serves_in.apply(classify_zone, axis=1)

    # get win proportions and convert to json
    zones = serves_in.groupby('Zone')['Point Winner'].value_counts().unstack()
    zones['Win Proportion'] = zones['host'].astype(str) + '/' + (zones['host'] + zones['guest']).astype(str)
    zones.reset_index(inplace=True)
    zones.columns.name = None
    zones = zones[['Zone', 'Win Proportion']]
    zones.to_json('serve_dist.json', orient='records')

In [5]:
path = '../../data/mens/Rudy Quan/combined.xlsx'
serve_zone_distribution(path)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/mens/Rudy Quan/combined.xlsx'

### Cj Work

In [7]:
player_name = 'Rudy Quan'

combined_data_shots = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Shots')
combined_data_points = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Points')
combined_data_games = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Games')
combined_data_sets = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Sets')
combined_data_stats = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Stats')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/mens/Rudy Quan/combined.xlsx'

In [ ]:
def classify_zone(df):
    
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            return 'Ad Wide'
        else:
            return 'Deuce Wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            return 'Ad Body'
        else:
            return 'Deuce Body'
    elif -52.5 < x < 52.5:
        if sign > 0:
            return 'Ad T'
        else:
            return 'Deuce T'
    else:
        return np.nan

In [ ]:
df_shots = combined_data_shots
df_points = combined_data_points

# only use matches with complete data
df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])] # UPDATE: Temporary fix

# add column for winner of the point
combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

serves = combined[(combined['Stroke'] == 'Serve') & (combined['Player'] == player_name)] # Added Player Name Filter
serves_in = serves[serves['Result'] == 'In']

# zone classification
serves_in['x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
serves_in['y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
serves_in['Zone'] = serves_in.apply(classify_zone, axis=1)

# get win proportions and convert to json
zones = serves_in.groupby('Zone')['Point Winner'].value_counts().unstack()
zones['Win Proportion'] = zones['host'].astype(str) + '/' + (zones['host'] + zones['guest']).astype(str)
zones.reset_index(inplace=True)
zones.columns.name = None
zones = zones[['Zone', 'Win Proportion']]
zones.to_json('serve_dist.json', orient='records')

In [ ]:
def serve_zone_distribution(df_shots, df_points):

    # only use matches with complete data
    df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])] # UPDATE: Temporary fix

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    serves = combined[(combined['Stroke'] == 'Serve') & (combined['Player'] == player_name)] # Added Player Name Filter
    serves_in = serves[serves['Result'] == 'In']

    # zone classification
    serves_in['x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in['y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in['Zone'] = serves_in.apply(classify_zone, axis=1)

    # get win proportions and convert to json
    zones = serves_in.groupby('Zone')['Point Winner'].value_counts().unstack()
    zones['Win Proportion'] = zones['host'].astype(str) + '/' + (zones['host'] + zones['guest']).astype(str)
    zones.reset_index(inplace=True)
    zones.columns.name = None
    zones = zones[['Zone', 'Win Proportion']]
    zones.to_json('serve_dist.json', orient='records')